This is just a little demo of how the code works. First import the module Data

In [1]:
from Data import *

We get access to the Data class type, which can be used to load the data and provide many support function. We now create an instance of Data and load from a sample csv file. Finally, we normalize the value.

In [2]:
D = Data()
D.loadfile("sample.csv")
D.normalize()

For raw access:

In [3]:
D.data[0][1]

[0.0,
 0.8421052631578949,
 0.191699604743083,
 0.572192513368984,
 0.2577319587628866,
 0.6195652173913043,
 0.6275862068965516,
 0.5738396624472574,
 0.28301886792452835,
 0.5930599369085174,
 0.37201365187713303,
 0.4552845528455285,
 0.9706959706959707,
 0.5613409415121255]

To get a copy of a point:

In [4]:
p = D.get(4)

To denote unknown values, denote it as None. For a point p with missing values, we can update the unknown feature f by calling D.update(p, f):

In [5]:
p[1][10] = None
p[1][1] = None
print("before",p)
D.update(p,10)
print("ater:",p)

before (4, [0.0, None, 0.36561264822134387, 0.8074866310160428, 0.5360824742268042, 0.5217391304347826, 0.6275862068965516, 0.4957805907172996, 0.49056603773584906, 0.4447949526813881, None, 0.4552845528455285, 0.6080586080586081, 0.325962910128388])
ater: (4, [0.0, None, 0.36561264822134387, 0.8074866310160428, 0.5360824742268042, 0.5217391304347826, 0.6275862068965516, 0.4957805907172996, 0.49056603773584906, 0.4447949526813881, 0.2593856655290102, 0.4552845528455285, 0.6080586080586081, 0.325962910128388])


You can get the binary vector of which features are known by calling bool_feature(p):

In [6]:
bool_feature(p)

[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

To partition the dataset into training and test sets, simply call D.split(r), r = ratio, which returns the test set and keeps D to be the training set

In [7]:
D_test = D.split(0.85)

print("D length:", len(D.data))
print("test length:", len(D_test.data))

D length: 27
test length: 151


Let's use k-means as our clustering algorithm. Calling D.cluster_K_means(k) will set the Data's clustering algorithm as k-means.

In [18]:
D.cluster_K_means(5)
D.clustering.cluster_centers_

prob: [0.2962962962962963, 0.18518518518518517, 0.2222222222222222, 0.18518518518518517, 0.1111111111111111]


array([[1.        , 0.60263158, 0.47875494, 0.56483957, 0.56829897,
        0.29619565, 0.1862069 , 0.08201477, 0.5495283 , 0.21766562,
        0.62670647, 0.18394309, 0.1510989 , 0.24304565],
       [0.        , 0.73578947, 0.25098814, 0.48235294, 0.26597938,
        0.37391304, 0.64206897, 0.57299578, 0.21509434, 0.47003155,
        0.32150171, 0.41300813, 0.77216117, 0.48573466],
       [0.5       , 0.27061404, 0.32575758, 0.48395722, 0.57474227,
        0.23550725, 0.35977011, 0.33298172, 0.58176101, 0.33491062,
        0.09143914, 0.40379404, 0.64224664, 0.11804565],
       [0.        , 0.76263158, 0.21857708, 0.67593583, 0.38350515,
        0.49347826, 0.62413793, 0.5164557 , 0.44150943, 0.37917981,
        0.37798635, 0.5300813 , 0.63809524, 0.7296719 ],
       [0.5       , 0.45964912, 0.07378129, 0.36898396, 0.29209622,
        0.4384058 , 0.35747126, 0.26090014, 0.57232704, 0.2807571 ,
        0.18856655, 0.56910569, 0.32112332, 0.29148835]])